Initiate esipy objects with dev configs and pyswagger cache then demonstrate operability by checking the status of the ccp api server

In [60]:
from config import *
from esipy import EsiApp, EsiClient, EsiSecurity, cache
import pandas as pd
import json, itertools
from pprint import PrettyPrinter

pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.expand_frame_repr', False)

f_cache = cache.FileCache(path="./f_cache")
# create the eve app interface
esiapp = EsiApp(cache=f_cache).get_latest_swagger

# init the security object
esisecurity = EsiSecurity(redirect_uri=ESI_CALLBACK,
                          client_id=ESI_CLIENT_ID,
                          secret_key=ESI_SECRET_KEY,
                          headers={'User-Agent': ESI_USER_AGENT})

# init the client
esiclient = EsiClient(security=esisecurity,
                      cache=f_cache,
                      headers={'User-Agent': ESI_USER_AGENT})

#  server_request_names = [k for k in esiapp.op.keys()] OR https://esi.evetech.net/
ccp_resp = esiclient.request(esiapp.op['get_status']())
print(f'Tranquility Status\nresp_code: {ccp_resp.status}\n{ccp_resp.data}')

def get_unique_region_ids():
    return pd.read_parquet("everefread")["regionID"].unique().tolist()


def req_all_region_order_pages():
    ops = []
    for id in get_unique_region_ids():
        ops.append(esiapp.op['get_markets_region_id_orders'](region_id=id))
    results = []
    for rq, rs in esiclient.multi_request(ops, raw_body_only=True):
        if rs.status == 200:
            pages = rs.header['X-Pages'][0]
            region = rq._Request__p.get('path')['region_id']
        results.append((region, pages))
    return results


def gen_region_market_requests(reg_id_pages:tuple):
    reg_id = reg_id_pages[0]
    pages = reg_id_pages[1]
    esiapp.op['get_markets_region_id_orders'](region_id=reg_id)
    operations = []
    for page in range(1, pages + 1):
        operations.append(esiapp.op['get_markets_region_id_orders'](
                region_id=reg_id, page=page))
    return operations

Tranquility Status
resp_code: 200
{'players': 16042, 'server_version': '2138646', 'start_time': <pyswagger.primitives._time.Datetime object at 0x000002A752DCA5C0>}


Next we request all region market ids and page counts.<br>
build a flat list of requests to send concurrently via esiclient<br>
send it and save resp in memory

In [61]:
ids_and_pages = req_all_region_order_pages()
req_bucket = list(map(gen_region_market_requests,ids_and_pages))
flat_req_bucket = list(itertools.chain(*req_bucket))
market_responses = esiclient.multi_request(flat_req_bucket, raw_body_only=True)

iterate over the json responses and build a dataframe with the unpacked orders and recovered region_id that goes with it

In [62]:
total_eve_orders = []
for rq, rs in market_responses:
    if rs.status == 200:
        for rec in json.loads(rs.raw):
            region = rq._Request__p.get('path')['region_id']
            total_eve_orders.append({'region':region,**rec})
pp = PrettyPrinter(width=180, compact=True)
pp.pprint(pd.DataFrame(total_eve_orders))

           region  duration  is_buy_order                issued    location_id  min_volume    order_id        price        range  system_id  type_id  volume_remain  volume_total
0        10000001        90         False  2022-10-07T11:40:36Z       60013867           1  6350667776    640000.00       region   30000019      380              1             2
1        10000001        90         False  2022-09-11T13:44:56Z       60012550           1  6336053255   1998000.00       region   30000109    21790              1             1
2        10000001        90         False  2022-10-20T17:48:08Z       60014452           1  6335605419    900000.00       region   30000112     2869            100           100
3        10000001        90         False  2022-07-31T20:59:26Z       60013024           1  6311280660         7.20       region   30000030      216          24000         70000
4        10000001        90         False  2022-10-06T13:05:32Z       60013867           1  6350176279 2000000

analyze the raw data in the dataframe and build our view from scratch.<br>
we will use the following columns:<br>
* region
* type_id
* price
* volume_remain
* location_id
* system_id
* is_buy_order

we will use the following derived columns:<br>
*sell_order_value = price * volume_remain*<br>

omit all buy orders from analysis<br>
iterate the view over all unique regions<br>
    get regional order count from view shape[0]<br>
    get most popular location_id from view as regional_hub with location_id.mode()<br>
    get most popular system_id from view as solar_system hub with system_id.mode()<br>
    get hub_market_value from location_id view as sum of sell_order_value<br>
    get hub_orders_count from location_id view with shape[0]<br>
    get hub_unique_items from location_id view as type_id.unique<br>

build a list(dict()) from the regional hub analysis<br>

In [63]:
total_eve_orders = pd.DataFrame(total_eve_orders)
total_eve_orders['sell_order_value'] = total_eve_orders.price * total_eve_orders.volume_remain
total_eve_sell_orders = total_eve_orders[total_eve_orders.is_buy_order == False]

systems = []

for reg_id in total_eve_sell_orders.region.unique():
    region_sell_orders = total_eve_sell_orders[total_eve_sell_orders.region == reg_id]
    regional_orders_count = region_sell_orders.shape[0]
    regional_hub_id = region_sell_orders.location_id.mode()[0]
    solar_system = region_sell_orders[region_sell_orders.location_id ==
                                        regional_hub_id].system_id.mode()[0]
    hub_sell_orders = region_sell_orders[region_sell_orders.location_id ==
                                        regional_hub_id]

    hub_market_value = hub_sell_orders.sell_order_value.sum()
    hub_orders_count = hub_sell_orders.shape[0]
    hub_unique_items = hub_sell_orders.type_id.unique().shape[0]


    systems.append({'region': int(reg_id), 'regional_hub': int(regional_hub_id),'unique_items': int(hub_unique_items), 'regional_order_count': int(regional_orders_count), 'hub_total_orders': int(hub_orders_count), 'hub_market_value': float(hub_market_value)})


use list(dict()) to build a dataframe from the regional hub analysis<br>
calculate the regional hub market orders share as a percentage of the total market orders<br>
get names to region_id, location_id from the eve api<br>
merge all responses to final dataframe<br>

In [64]:
pp = PrettyPrinter(width=150,compact=True)
df_orders = pd.DataFrame(systems)
df_orders['hub_regional_order_ratio'] = round((df_orders.hub_total_orders / df_orders.regional_order_count) * 100,1)
id_list = df_orders.regional_hub.to_list()
req = esiapp.op['post_universe_names'](ids=id_list)
ccp_resp = esiclient.request(req)
with_sta_names = pd.DataFrame().from_records(ccp_resp.data).merge(
    df_orders, left_on='id', right_on='regional_hub').drop(columns=['id', 'category'])

id_list = with_sta_names.region.to_list()
req = esiapp.op['post_universe_names'](ids=id_list)
ccp_resp = esiclient.request(req)
pp.pprint(pd.DataFrame().from_records(ccp_resp.data).merge( with_sta_names, left_on='id', right_on='region').sort_values('hub_total_orders', ascending=False).drop(columns=['id', 'category']).head(10))


          name_x                                             name_y    region  regional_hub  unique_items  regional_order_count  hub_total_orders    hub_market_value  hub_regional_order_ratio
0      The Forge     Jita IV - Moon 4 - Caldari Navy Assembly Plant  10000002      60003760         14226                198039            166931  170939133325952.88                     84.30
28        Domain         Amarr VIII (Oris) - Emperor Family Academy  10000043      60008494          9476                 95384             52346    7902528142589.25                     54.90
1    Sinq Laison  Dodixie IX - Moon 20 - Federation Navy Assembl...  10000032      60011866          8112                 65610             31168  135299161206341.81                     47.50
33      Heimatar           Rens VI - Moon 8 - Brutor Tribe Treasury  10000030      60004588          6431                 47590             18176    2166499100089.66                     38.20
2     Metropolis    Hek VIII - Moon 12 -